In [268]:
print("Working")

Working


In [269]:
import json
with open('twitter_credentials.json') as f:
    credentials = json.load(f)

In [270]:
import re
# pip install tweepy
import tweepy

auth = tweepy.OAuthHandler(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'])
auth.set_access_token(credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'])

api = tweepy.API(auth)

In [271]:
def clear_tweet(tweet, toRemove):
    return tweet.replace(toRemove, "")

In [272]:
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0

    def on_status(self, status):
        # don't get retweets
        if not status.retweeted and 'RT @' not in status.text:
            self.num_tweets += 1
            # stop at 20 tweets
            if self.num_tweets < 20:
                myjson = json.loads(json.dumps(status._json))
                print("======== TWEET ========")
                # get tweet text
                if 'extended_tweet' not in myjson:
                    tweet = myjson['text']
                else:
                    tweet = myjson['extended_tweet']['full_text']
                # clean tweet from urls
                tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet)
                print(tweet)
                return True
            else:
                print(self.num_tweets)
                return False        
    
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

        # returning non-False reconnects the stream, with backoff.

In [273]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(track=['trump'], languages=["en"], async=True)

======== TWEET ========
 Russians pumped propaganda into the UK and they fell for it just like the Americans did with Trump. Sad.
======== TWEET ========
 I quote Steven Choi in my piece on the citizenship question trial 
======== TWEET ========
Making ready to do my "happy dance"!
======== TWEET ========
 Hi the unroll you asked for: Thread by  "I just broke all my own rules And yelled at my Trump loving Father on the phone, hung up on him, too I can’t help it I t […]" #ParadiseFire #ParadiseLost 
Talk to you soon. 🤖
======== TWEET ========
...and that people need ID to buy groceries.

#25thamendment #UnfitForOffice
======== TWEET ========
 I can't stand Trump. But Hillary running is the worst idea for the Dems. They need someone young, with a platform that people can get behind. They need new blood.
======== TWEET ========
One-on-one confrontation is what they need because a street brawl will only be good optics for the left. After a riot by Antifa, they need to be followed home by p